In [ ]:
!git clone https://github.com/megvii-research/KPAFlow.git

Cloning into 'KPAFlow'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 46 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 30.27 KiB | 1.12 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Create dataset folder for Sintel
!mkdir -p /content/Sintel

# Download MPI Sintel dataset (this link is correct for the training images)#
!wget -P /content/Sintel/ http://files.is.tue.mpg.de/sintel/MPI-Sintel-complete.zip

# Unzip the Sintel dataset (make sure the file name matches what you downloaded)
!unzip /content/drive/MyDrive/Sintel/MPI-Sintel-complete.zip -d /content/Sintel/



Streaming output truncated to the last 5000 lines.
  inflating: /content/Sintel/training/invalid/ambush_2/frame_0016.png  
  inflating: /content/Sintel/training/invalid/ambush_2/frame_0011.png  
  inflating: /content/Sintel/training/invalid/ambush_2/frame_0007.png  
  inflating: /content/Sintel/training/invalid/ambush_2/frame_0009.png  
   creating: /content/Sintel/training/invalid/cave_2/
  inflating: /content/Sintel/training/invalid/cave_2/frame_0028.png  
  inflating: /content/Sintel/training/invalid/cave_2/frame_0021.png  
  inflating: /content/Sintel/training/invalid/cave_2/frame_0026.png  
  inflating: /content/Sintel/training/invalid/cave_2/frame_0013.png  
  inflating: /content/Sintel/training/invalid/cave_2/frame_0014.png  
  inflating: /content/Sintel/training/invalid/cave_2/frame_0005.png  
  inflating: /content/Sintel/training/invalid/cave_2/frame_0002.png  
  inflating: /content/Sintel/training/invalid/cave_2/frame_0037.png  
  inflating: /content/Sintel/training/invalid/c

In [ ]:
!cp /content/Sintel/MPI-Sintel-complete.zip /content/drive/MyDrive/Sintel/

In [ ]:
!pwd


/content/KPAFlow


In [ ]:
%cd /content/KPAFlow

/content/KPAFlow


In [ ]:
!mkdir --help

Usage: mkdir [OPTION]... DIRECTORY...
Create the DIRECTORY(ies), if they do not already exist.

Mandatory arguments to long options are mandatory for short options too.
  -m, --mode=MODE   set file mode (as in chmod), not a=rwx - umask
  -p, --parents     no error if existing, make parent directories as needed
  -v, --verbose     print a message for each created directory
  -Z                   set SELinux security context of each created directory
                         to the default type
      --context[=CTX]  like -Z, or if CTX is specified then set the SELinux
                         or SMACK security context to CTX
      --help     display this help and exit
      --version  output version information and exit

GNU coreutils online help: <https://www.gnu.org/software/coreutils/>
Full documentation <https://www.gnu.org/software/coreutils/mkdir>
or available locally via: info '(coreutils) mkdir invocation'


In [ ]:
!mkdir -p datasets/Sintel/training/clean

In [ ]:
!cp -r /content/Sintel/training/* datasets/Sintel/training/

In [ ]:
import torch
print(torch.cuda.device_count())


1


In [ ]:
import torch

# Check if CUDA (GPU) is available; otherwise, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



Using device: cpu


In [ ]:
ls "/content/drive/MyDrive/Communication (My notes)/"


'Amplitude modulation .pdf'   'noise in analog communication.pdf'    'Random Variable.pdf'
'angle modulation.pdf'        'Noise in digital communication .pdf'   Sampling.pdf
'Bandpass Transmission .pdf'  'Poisson distribution.pdf'             'transmitter and reciever.pdf'
 KPAFlow/                      Quantization.pdf


In [ ]:
cp -r "/content/drive/MyDrive/Communication (My notes)/KPAFlow" /content/


^C


In [ ]:
!python train.py --name KPAFlow --stage sintel --dataset /content/Sintel --num_steps 100000 --batch_size 6 --gpus 0 --lr 0.00002


2024-11-14 08:15:07.435840: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-14 08:15:07.453878: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-14 08:15:07.475246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-14 08:15:07.481737: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 08:15:07.497159: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
!pwd

/content/KPAFlow


In [ ]:
import torch
import torchvision.ops
from torch import nn

class DeformableConv2d(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride=1,
                 padding=1,
                 bias=False):

        super(DeformableConv2d, self).__init__()

        self.padding = padding

        self.offset_conv = nn.Conv2d(in_channels,
                                     2 * kernel_size * kernel_size,
                                     kernel_size=kernel_size,
                                     stride=stride,
                                     padding=self.padding,
                                     bias=True)

        nn.init.constant_(self.offset_conv.weight, 0.)
        nn.init.constant_(self.offset_conv.bias, 0.)

        self.modulator_conv = nn.Conv2d(in_channels,
                                     1 * kernel_size * kernel_size,
                                     kernel_size=kernel_size,
                                     stride=stride,
                                     padding=self.padding,
                                     bias=True)

        nn.init.constant_(self.modulator_conv.weight, 0.)
        nn.init.constant_(self.modulator_conv.bias, 0.)

        self.regular_conv = nn.Conv2d(in_channels=in_channels,
                                      out_channels=out_channels,
                                      kernel_size=kernel_size,
                                      stride=stride,
                                      padding=self.padding,
                                      bias=bias)

    def forward(self, x):
        h, w = x.shape[2:]
        max_offset = max(h, w)/4.

        offset = self.offset_conv(x).clamp(-max_offset, max_offset)
        modulator = 2. * torch.sigmoid(self.modulator_conv(x))

        x = torchvision.ops.deform_conv2d(input=x,
                                          offset=offset,
                                          weight=self.regular_conv.weight,
                                          bias=self.regular_conv.bias,
                                          padding=self.padding,
                                          mask=modulator
                                          )
        return x

In [ ]:
!python evaluate.py --model=/content/KPAFlow/weights/KPAFlow-CT2S.pth --dataset=sintel --mixed_precision




----- Model: KPA-Flow -----
DataParallel(
  (module): KPAFlow(
    (fnet): BasicEncoder(
      (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (relu1): ReLU(inplace=True)
      (layer1): Sequential(
        (0): ResidualBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (relu): ReLU(inplace=True)
          (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        )
        (1): ResidualBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (relu): 

In [ ]:
!pwd

/content/KPAFlow


In [ ]:
%cd KPAFlow

/content/KPAFlow


In [ ]:
import sys
sys.path.append('core')

from PIL import Image
import argparse
import os
import time
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
sys.path.insert(1, './core/')
import datasets
from utils import flow_viz
from utils import frame_utils

from KPAFlow import KPAFlow

from utils.utils import InputPadder, forward_interpolate


@torch.no_grad()
def create_sintel_submission(model, iters=32, warm_start=False, output_path='sintel_submission'):
    """ Create submission for the Sintel leaderboard """
    model.eval()
    for dstype in ['clean', 'final']:
        test_dataset = datasets.MpiSintel(split='test', aug_params=None, dstype=dstype)

        flow_prev, sequence_prev = None, None
        for test_id in range(len(test_dataset)):
            image1, image2, (sequence, frame) = test_dataset[test_id]
            # if sequence != sequence_prev:
            #     flow_prev = None

            if (sequence != sequence_prev) or (dstype == 'final' and sequence in ['market_4', ]) or dstype == 'clean':
                flow_prev = None

            padder = InputPadder(image1.shape)
            image1, image2 = padder.pad(image1[None].cuda(), image2[None].cuda())

            flow_low, flow_pr = model(image1, image2, iters=iters, flow_init=flow_prev, test_mode=True)
            flow = padder.unpad(flow_pr[0]).permute(1, 2, 0).cpu().numpy()

            if warm_start:
                flow_prev = forward_interpolate(flow_low[0])[None].cuda()

            output_dir = os.path.join(output_path, dstype, sequence)
            output_file = os.path.join(output_dir, 'frame%04d.flo' % (frame+1))

            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            frame_utils.writeFlow(output_file, flow)
            sequence_prev = sequence


@torch.no_grad()
def create_kitti_submission(model, iters=24, output_path='kitti_submission'):
    """ Create submission for the Sintel leaderboard """
    model.eval()
    test_dataset = datasets.KITTI(split='testing', aug_params=None)

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for test_id in range(len(test_dataset)):
        image1, image2, (frame_id, ) = test_dataset[test_id]
        padder = InputPadder(image1.shape, mode='kitti')
        image1, image2 = padder.pad(image1[None].cuda(), image2[None].cuda())

        _, flow_pr = model(image1, image2, iters=iters, test_mode=True)
        flow = padder.unpad(flow_pr[0]).permute(1, 2, 0).cpu().numpy()

        output_filename = os.path.join(output_path, frame_id)
        frame_utils.writeFlowKITTI(output_filename, flow)


@torch.no_grad()
def validate_chairs(model, iters=24):
    """ Perform evaluation on the FlyingChairs (test) split """
    model.eval()
    epe_list = []

    val_dataset = datasets.FlyingChairs(split='validation')
    for val_id in range(len(val_dataset)):
        image1, image2, flow_gt, _ = val_dataset[val_id]
        image1 = image1[None].cuda()
        image2 = image2[None].cuda()

        _, flow_pr = model(image1, image2, iters=iters, test_mode=True)
        epe = torch.sum((flow_pr[0].cpu() - flow_gt)**2, dim=0).sqrt()
        epe_list.append(epe.view(-1).numpy())

    epe = np.mean(np.concatenate(epe_list))
    print("Validation Chairs EPE: %f" % epe)
    return {'chairs': epe}


@torch.no_grad()
def validate_sintel(model, iters=32):
    """ Peform validation using the Sintel (train) split """
    model.eval()
    results = {}
    for dstype in ['clean', 'final']:
        val_dataset = datasets.MpiSintel(split='training', dstype=dstype)
        epe_list = []

        for val_id in range(len(val_dataset)):
            image1, image2, flow_gt, _ = val_dataset[val_id]
            image1 = image1[None].cuda()
            image2 = image2[None].cuda()

            padder = InputPadder(image1.shape)
            image1, image2 = padder.pad(image1, image2)

            flow_low, flow_pr = model(image1, image2, iters=iters, test_mode=True)
            flow = padder.unpad(flow_pr[0]).cpu()

            epe = torch.sum((flow - flow_gt)**2, dim=0).sqrt()
            epe_list.append(epe.view(-1).numpy())

        epe_all = np.concatenate(epe_list)
        epe = np.mean(epe_all)
        px1 = np.mean(epe_all<1)
        px3 = np.mean(epe_all<3)
        px5 = np.mean(epe_all<5)

        print("Validation (%s) EPE: %f, 1px: %f, 3px: %f, 5px: %f" % (dstype, epe, px1, px3, px5))
        results[dstype] = np.mean(epe_list)

    return results


@torch.no_grad()
def validate_kitti(model, iters=24):
    """ Peform validation using the KITTI-2015 (train) split """
    model.eval()
    val_dataset = datasets.KITTI(split='training')

    out_list, epe_list = [], []
    for val_id in range(len(val_dataset)):
        image1, image2, flow_gt, valid_gt = val_dataset[val_id]
        image1 = image1[None].cuda()
        image2 = image2[None].cuda()

        padder = InputPadder(image1.shape, mode='kitti')
        image1, image2 = padder.pad(image1, image2)

        flow_low, flow_pr = model(image1, image2, iters=iters, test_mode=True)
        flow = padder.unpad(flow_pr[0]).cpu()

        epe = torch.sum((flow - flow_gt)**2, dim=0).sqrt()
        mag = torch.sum(flow_gt**2, dim=0).sqrt()

        epe = epe.view(-1)
        mag = mag.view(-1)
        val = valid_gt.view(-1) >= 0.5

        out = ((epe > 3.0) & ((epe/mag) > 0.05)).float()
        epe_list.append(epe[val].mean().item())
        out_list.append(out[val].cpu().numpy())

    epe_list = np.array(epe_list)
    out_list = np.concatenate(out_list)

    epe = np.mean(epe_list)
    f1 = 100 * np.mean(out_list)

    print("Validation KITTI: %f, %f" % (epe, f1))
    return {'kitti-epe': epe, 'kitti-f1': f1}



parser = argparse.ArgumentParser()
parser.add_argument('--model', help="restore checkpoint")
parser.add_argument('--dataset', help="dataset for evaluation")
parser.add_argument('--small', action='store_true', help='use small model')
parser.add_argument('--mixed_precision', action='store_true', help='use mixed precision')
parser.add_argument('--alternate_corr', action='store_true', help='use efficent correlation implementation')
args = parser.parse_args()

model = torch.nn.DataParallel(KPAFlow(args))
model.load_state_dict(torch.load(args.model))

# model.cuda()
# model.eval()

# create_sintel_submission(model.module, warm_start=True)
# create_kitti_submission(model.module)

# with torch.no_grad():
#     if args.dataset == 'chairs':
#         validate_chairs(model.module)

#     elif args.dataset == 'sintel':
#         validate_sintel(model.module)

#     elif args.dataset == 'kitti':
#         validate_kitti(model.module)




usage: colab_kernel_launcher.py [-h] [--model MODEL] [--dataset DATASET] [--small]
                                [--mixed_precision] [--alternate_corr]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-21a7d831-29d3-4dbf-99ab-a0291cefbaeb.json


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!zip -r new.zip /content/sintel_visualization/


  adding: content/sintel_visualization/ (stored 0%)
  adding: content/sintel_visualization/flow_0101.png (deflated 0%)
  adding: content/sintel_visualization/flow_0344.png (deflated 0%)
  adding: content/sintel_visualization/flow_0569.png (deflated 0%)
  adding: content/sintel_visualization/flow_0869.png (deflated 0%)
  adding: content/sintel_visualization/flow_0271.png (deflated 0%)
  adding: content/sintel_visualization/flow_0013.png (deflated 0%)
  adding: content/sintel_visualization/flow_0400.png (deflated 0%)
  adding: content/sintel_visualization/flow_0939.png (deflated 1%)
  adding: content/sintel_visualization/flow_0744.png (deflated 0%)
  adding: content/sintel_visualization/flow_0898.png (deflated 0%)
  adding: content/sintel_visualization/flow_0159.png (deflated 0%)
  adding: content/sintel_visualization/flow_0657.png (deflated 0%)
  adding: content/sintel_visualization/flow_0129.png (deflated 0%)
  adding: content/sintel_visualization/flow_0309.png (deflated 0%)
  adding: 